In [ ]:
import sqlalchemy as sa
import sqlalchemy.orm as so
from sqlalchemy.orm import DeclarativeBase, Session
from pathlib import Path
import pandas as pd
import tempfile
import logging
from orm_loader.tables.base import CSVLoadableTableInterface  

logging.basicConfig(level=logging.INFO)

class Base(DeclarativeBase):
    pass

engine = sa.create_engine("sqlite:///test.db", echo=False, future=True)
Base.metadata.bind = engine


class TestTable(Base, CSVLoadableTableInterface):
    __tablename__ = "test_table"

    id: so.Mapped[int] = so.mapped_column(primary_key=True)
    name: so.Mapped[str] = so.mapped_column(nullable=False)

Base.metadata.create_all(engine)

tmp = Path(tempfile.mkdtemp())

csv_initial = tmp / "test_table.csv"
csv_replace = tmp / "test_table_replace.csv"

pd.DataFrame(
    [
        {"id": 1, "name": "alpha"},
        {"id": 2, "name": "beta"},
        {"id": 3, "name": "gamma"},
    ]
).to_csv(csv_initial, index=False, sep="\t")

pd.DataFrame(
    [
        {"id": 2, "name": "beta_updated"},
        {"id": 3, "name": "gamma_updated"},
    ]
).to_csv(csv_replace, index=False, sep="\t")

csv_replace = tmp / "test_table.csv"
pd.DataFrame(
    [
        {"id": 2, "name": "beta_updated"},
        {"id": 3, "name": "gamma_updated"},
    ]
).to_csv(csv_replace, index=False, sep="\t")


In [6]:
with Session(engine) as session:
    inserted = TestTable.load_csv(
        session,
        csv_initial,
        dedupe=False,
    )
    session.commit()

    rows = session.execute(
        sa.select(TestTable).order_by(TestTable.id)
    ).scalars().all()

rows


INFO:orm_loader.tables.data.ingestion:Detected encoding utf-8 for file test_table.csv
INFO:orm_loader.tables.data.ingestion:Detected delimiter '	' for file test_table.csv


[<__main__.TestTable at 0x10b35dd00>, <__main__.TestTable at 0x10b9f3b30>]

In [7]:
with Session(engine) as session:
    rows = session.execute(
        sa.select(TestTable).order_by(TestTable.id)
    ).scalars().all()
rows

[<__main__.TestTable at 0x10bc2d070>, <__main__.TestTable at 0x10bc2d040>]

In [8]:
with Session(engine) as session:
    replaced = TestTable.replace_from_csv(
        session,
        csv_replace,
    )
    session.commit()

    rows = session.execute(
        sa.select(TestTable).order_by(TestTable.id)
    ).scalars().all()

rows


INFO:orm_loader.tables.base.loadable_table:Replacing data in test_table from test_table.csv
INFO:orm_loader.tables.data.ingestion:Detected encoding utf-8 for file test_table.csv
INFO:orm_loader.tables.data.ingestion:Detected delimiter '	' for file test_table.csv


[<__main__.TestTable at 0x10bc2e0c0>, <__main__.TestTable at 0x10bc2e060>]

In [9]:
with engine.connect() as conn:
    tables = conn.execute(
        sa.text(
            "SELECT name FROM sqlite_master WHERE type='table'"
        )
    ).fetchall()

tables

[('test_table',)]